# Logistic Regression - MultinomialClassification ( MNIST )
softmax를 사용한 로지스틱 다중 분류 예제.   
( 딥러닝의 방식인 층을 사용하지 않고 로지스틱회귀로만 푸는 예제 )
> 출처
1. Andrew NG
2. Sung Kim

데이터 출처 : MNIST

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

## 1. 데이터 불러오기 및 파라미터 설정

In [2]:
# mnist 데이터 불러오기.
( train_x, train_y ), (test_x, test_y ) = tf.keras.datasets.mnist.load_data()

# MNIST 데이터 파라미터 설정
num_classes = 10 # 0부터 9까지의 숫자
num_features = 784 # 28*28

# 하이퍼파라미터
learning_rate = 0.01
num_epochs = 100
batch_size = 256
num_data = train_x.shape[0]

# 딥러닝이 아닌 간단한 로지스틱 회귀를 위해서 2차원(이미지)를 1차원으로 바꿔준다.
train_X = tf.constant(train_x, dtype=tf.float32)
train_X = tf.reshape(train_X, [-1,num_features])
train_Y = tf.one_hot(train_y, depth=10) # y값 one-hot encoding

test_X = tf.constant(test_x, dtype=tf.float32)
test_X = tf.reshape(test_X, [-1,num_features])
test_Y = tf.one_hot(test_y, depth=10) # y값 one-hot encoding

# 데이터 셔플링 및 배치화
train_data = tf.data.Dataset.from_tensor_slices((train_X, train_Y))
train_data = train_data.repeat().shuffle(5000).batch(batch_size)
iterater = train_data.__iter__()

# 파라미터 설정
W = tf.Variable(tf.random.normal([num_features,num_classes]))
b = tf.Variable(tf.random.normal([num_classes]))


## 2. 함수 정의

In [3]:
# 가설 함수
def hypothesis(x):
    return tf.nn.softmax(tf.matmul(x,W)+b)

# 비용 함수 cross-entropy를 사용한다.
def cost_function(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.) # log(0) error를 피하기 위함.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred),axis=1))

# Gradient Descent
def run_optimizer(x,y):
    with tf.GradientTape() as tape:
        predict = hypothesis(x)
        cost = cost_function(y, predict)

    dJ_dW,dJ_db = tape.gradient(cost,[W,b])
    W.assign_sub(learning_rate*dJ_dW)
    b.assign_sub(learning_rate*dJ_db)
    
def accuracy(y_true, y_pred):
    predict = tf.argmax(y_pred, axis=1)
    y_true = tf.argmax(y_true,axis=1)
    correct = tf.cast(tf.equal(predict,y_true),dtype=tf.float32)
    accuracy = tf.reduce_mean(correct)
    return accuracy

## 3. 훈련 및 평가

In [4]:
cost_log = []
accuracy_log =[]
for epoch in range(num_epochs):

    for i in range(int(num_data/batch_size)):
        x_batch, y_batch = iterater.get_next()

        run_optimizer(x_batch, y_batch)

    cost_value = cost_function(train_Y, hypothesis(train_X))
    cost_log.append(cost_value)
    train_accuracy = accuracy(train_Y, hypothesis(train_X))
    test_accuracy = accuracy(test_Y, hypothesis(test_X))
    accuracy_log.append(test_accuracy)
    print("epoch:",epoch,"cost:",cost_value.numpy(),"train_accuracy:",train_accuracy.numpy(),"test_accuracy:",test_accuracy.numpy())


epoch: 0 cost: 18.304104 train_accuracy: 0.11615 test_accuracy: 0.1132
epoch: 1 cost: 16.978436 train_accuracy: 0.18 test_accuracy: 0.1752
epoch: 2 cost: 16.067318 train_accuracy: 0.2239 test_accuracy: 0.2174
epoch: 3 cost: 15.362184 train_accuracy: 0.25816667 test_accuracy: 0.2508
epoch: 4 cost: 14.571553 train_accuracy: 0.29608333 test_accuracy: 0.2935
epoch: 5 cost: 14.14193 train_accuracy: 0.31703332 test_accuracy: 0.3137
epoch: 6 cost: 13.68761 train_accuracy: 0.33868334 test_accuracy: 0.334
epoch: 7 cost: 13.386515 train_accuracy: 0.35326666 test_accuracy: 0.3502
epoch: 8 cost: 13.187885 train_accuracy: 0.36293334 test_accuracy: 0.3612
epoch: 9 cost: 12.955848 train_accuracy: 0.37425 test_accuracy: 0.3733
epoch: 10 cost: 12.859248 train_accuracy: 0.37905 test_accuracy: 0.379
epoch: 11 cost: 12.672297 train_accuracy: 0.38798332 test_accuracy: 0.386
epoch: 12 cost: 12.562221 train_accuracy: 0.39316666 test_accuracy: 0.387
epoch: 13 cost: 12.28031 train_accuracy: 0.4069 test_accurac

In [1]:
# 비용함수
plt.plot(np.arange(0,100),cost_log)
plt.xlabel('epoch')
plt.ylabel('cost_value')
plt.show()

NameError: name 'plt' is not defined